In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\MarcellNemeth\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


CICIDS

In [166]:
data_df = pd.read_excel('measurements/CD_delta_vs_ro_chp_tabs_cicids.xlsx', sheet_name='corr_measurements')

In [167]:
# Create the reversed pairs
reversed_df = data_df.rename(columns={'feat_1': 'feat_2', 'feat_2': 'feat_1', 'feat_1_drift': 'feat_2_drift', 'feat_2_drift': 'feat_1_drift', 'feat_1_cd_score': 'feat_2_cd_score', 'feat_2_cd_score': 'feat_1_cd_score'})

# Concatenate the original dataframe with the reversed pairs
data_df = pd.concat([data_df, reversed_df], ignore_index=True)

# Display the resulting dataframe
data_df

,feat_1,feat_2,ref_corr,analysis_corr,corr_delta,feat_1_cd_score,feat_2_cd_score,delta_cd_score,feat_1_drift,feat_2_drift,is_single_drift,feat_pair_drift
0,Init_Win_bytes_forward,Init_Win_bytes_backward,0.581335,0.039265,0.620600,0.622063,0.262135,0.359928,1,1,0,1
1,Init_Win_bytes_forward,Fwd IAT Min,0.016107,0.037673,0.053780,0.622063,0.250770,0.371293,1,1,0,1
2,Init_Win_bytes_forward,Bwd Packet Length Max,0.367913,0.045976,0.321937,0.622063,0.109915,0.512148,1,0,1,0
3,Init_Win_bytes_forward,Bwd Packets/s,0.067652,0.005138,0.062514,0.622063,0.091678,0.530385,1,0,1,0
4,Init_Win_bytes_forward,Flow IAT Min,0.037748,0.112909,0.075161,0.622063,0.089419,0.532644,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
337,Bwd Packet Length Min,Min Packet Length,0.836438,0.940416,0.103978,0.258648,0.551775,0.293128,1,1,0,1
338,Down/Up Ratio,Min Packet Length,0.025521,0.127896,0.102374,0.112505,0.551775,0.439270,0,1,1,0
339,Bwd Packet Length Min,URG Flag Count,0.050013,0.028802,0.021210,0.258648,0.070690,0.187957,1,0,1,0
340,Down/Up Ratio,URG Flag Count,0.014657,0.008563,0.023220,0.112505,0.070690,0.041815,0,0,0,0


In [219]:
m = data_df['feat_1'].nunique()

In [221]:
num_monitored_features = 10
num_non_monitored_features = m - num_monitored_features

In [223]:
# choose num_monitored_features random features from the values of the 'feat_1' column
monitored_features = np.random.choice(data_df['feat_1'].unique(), num_monitored_features, replace=False)
non_monitored_features = np.setdiff1d(data_df['feat_1'].unique(), monitored_features)

In [225]:
num_monitored_features_drifting = data_df[data_df['feat_1'].isin(monitored_features)].drop_duplicates(subset=['feat_1']).feat_1_drift.sum()

In [229]:
num_non_monitored_features_drifting = data_df[data_df['feat_1'].isin(non_monitored_features)].drop_duplicates(subset=['feat_1']).feat_1_drift.sum()

In [231]:
non_monitored_ratio = num_non_monitored_features / m

In [233]:
all_features_drifting = data_df[data_df['feat_1_drift']==1].drop_duplicates(subset=['feat_1']).feat_1.unique()

In [235]:
all_features_non_drifting = data_df[data_df['feat_1_drift']==0].drop_duplicates(subset=['feat_1']).feat_1.unique()

In [246]:
corr_threshold = 0.5

In [247]:
pred_features_drifting = data_df[(data_df['feat_1'].isin(monitored_features)) & (data_df['feat_2'].isin(non_monitored_features)) & (data_df['ref_corr']>=corr_threshold)].drop_duplicates(subset=['feat_2']).feat_2.unique()

In [249]:
tp_features_drifting = set(pred_features_drifting).intersection(all_features_drifting)

In [280]:
pred_features_non_drifting = set(non_monitored_features) - set(pred_features_drifting)

In [281]:
tn_features_drifting = set(pred_features_non_drifting).intersection(all_features_non_drifting)

In [282]:
num_tn_features_drifting = len(tn_features_drifting)

In [284]:
num_tp_features_drifting = len(tp_features_drifting)

In [286]:
num_pred_features_drifting_non_monitored = len(set(pred_features_drifting).intersection(non_monitored_features))

In [287]:
num_pred_features_drifting_monitored = len(set(pred_features_drifting).intersection(monitored_features))

In [288]:
# create dataframe having the following columns: dataset, corr_threshold, num_monitored_features, num_non_monitored_features, num_tp_features_drifting, num_non_monitored_features_drifting, num_pred_features_drifting_non_monitored
results_df_non_monitored = pd.DataFrame(columns=['dataset', 'corr_threshold', 'num_monitored_features', 'num_non_monitored_features', 'non_monitored_ratio', 'num_tp_features_drifting', 'num_tn_features_drifting', 'num_non_monitored_features_drifting', 'num_pred_features_drifting_non_monitored'])

In [289]:
dataset_name = 'CICIDS'

In [319]:
# put all above into a function

def compute_results(data_df, dataset_name, fold, corr_threshold, num_monitored_features):
    m = data_df['feat_1'].nunique()
    num_non_monitored_features = m - num_monitored_features
    non_monitored_ratio = num_non_monitored_features / m

    monitored_features = np.random.choice(data_df['feat_1'].unique(), num_monitored_features, replace=False)
    non_monitored_features = np.setdiff1d(data_df['feat_1'].unique(), monitored_features)

    # num_monitored_features_drifting = data_df[data_df['feat_1'].isin(monitored_features)].drop_duplicates(subset=['feat_1']).feat_1_drift.sum()

    num_non_monitored_features_drifting = data_df[data_df['feat_1'].isin(non_monitored_features)].drop_duplicates(subset=['feat_1']).feat_1_drift.sum()

    all_features_drifting = data_df[data_df['feat_1_drift']==1].drop_duplicates(subset=['feat_1']).feat_1.unique()

    all_features_non_drifting = data_df[data_df['feat_1_drift']==0].drop_duplicates(subset=['feat_1']).feat_1.unique()

    pred_features_drifting = data_df[(data_df['feat_1'].isin(monitored_features)) & (data_df['feat_2'].isin(non_monitored_features)) & (data_df['ref_corr']>=corr_threshold)].drop_duplicates(subset=['feat_2']).feat_2.unique()

    tp_features_drifting = set(pred_features_drifting).intersection(all_features_drifting)

    pred_features_non_drifting = set(non_monitored_features) - set(pred_features_drifting)

    tn_features_drifting = set(pred_features_non_drifting).intersection(all_features_non_drifting)

    num_tn_features_drifting = len(tn_features_drifting)

    num_tp_features_drifting = len(tp_features_drifting)

    num_pred_features_drifting_non_monitored = len(set(pred_features_drifting).intersection(non_monitored_features))

    results_df_non_monitored = pd.DataFrame(columns=['dataset', 'corr_threshold', 'num_monitored_features', 'fold', 'num_non_monitored_features', 'non_monitored_ratio', 'num_tp_features_drifting', 'num_tn_features_drifting', 'num_non_monitored_features_drifting', 'num_pred_features_drifting_non_monitored'])

    results_df_non_monitored = results_df_non_monitored.append({'dataset': dataset_name,
                                                                'corr_threshold': corr_threshold,
                                                                'num_monitored_features': num_monitored_features,
                                                                'fold': fold,
                                                                'num_non_monitored_features': num_non_monitored_features,
                                                                'non_monitored_ratio': non_monitored_ratio,
                                                                'num_tp_features_drifting': num_tp_features_drifting,
                                                                'num_tn_features_drifting': num_tn_features_drifting,
                                                                'num_non_monitored_features_drifting': num_non_monitored_features_drifting,
                                                                'num_pred_features_drifting_non_monitored': num_pred_features_drifting_non_monitored}, ignore_index=True)
    
    try:
        results_df_non_monitored['accuracy'] = (results_df_non_monitored['num_tp_features_drifting'] + results_df_non_monitored['num_tn_features_drifting']) / results_df_non_monitored['num_non_monitored_features']
        results_df_non_monitored['recall'] = results_df_non_monitored['num_tp_features_drifting'] / results_df_non_monitored['num_non_monitored_features_drifting']
        results_df_non_monitored['precision'] = results_df_non_monitored['num_tp_features_drifting'] / num_pred_features_drifting_non_monitored
        results_df_non_monitored['f1_score'] = 2 * (results_df_non_monitored['precision'] * results_df_non_monitored['recall']) / (results_df_non_monitored['precision'] + results_df_non_monitored['recall'])
    except ZeroDivisionError:
        results_df_non_monitored['accuracy'] = 0
        results_df_non_monitored['recall'] = 0
        results_df_non_monitored['precision'] = 0
        results_df_non_monitored['f1_score'] = 0
    
    return results_df_non_monitored

In [320]:
# run a 10 fold cross validation, append the results to a final_measurements dataframe
final_measurements = pd.DataFrame(columns=['dataset', 'corr_threshold', 'num_monitored_features', 'fold', 'num_non_monitored_features', 'non_monitored_ratio', 'num_tp_features_drifting', 'num_tn_features_drifting', 'num_non_monitored_features_drifting', 'num_pred_features_drifting_non_monitored', 'accuracy', 'recall', 'precision', 'f1_score'])

for corr_threshold in np.arange(0.1, 1.0, 0.1):
    for num_monitored_features in range(int(m).bit_length()):
        num_monitored_features_val = int(m / (2 ** num_monitored_features))
        if num_monitored_features_val < 1:
            break
        for fold in range(10):
            results_df_non_monitored = compute_results(data_df, dataset_name, fold, corr_threshold, num_monitored_features_val)
            final_measurements = final_measurements.append(results_df_non_monitored, ignore_index=True)

C:\Users\MARCEL~1\AppData\Local\Temp/ipykernel_17852/420335379.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df_non_monitored = results_df_non_monitored.append({'dataset': dataset_name,
C:\Users\MARCEL~1\AppData\Local\Temp/ipykernel_17852/1495189836.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_measurements = final_measurements.append(results_df_non_monitored, ignore_index=True)
C:\Users\MARCEL~1\AppData\Local\Temp/ipykernel_17852/420335379.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df_non_monitored = results_df_non_monitored.append({'dataset': dataset_name,
C:\Users\MARCEL~1\AppData\Local\Temp/ipykernel_17852/1495189836.py:11: FutureWarning: The frame.append method is deprecate

In [325]:
final_measurements.groupby(['corr_threshold', 'num_monitored_features']).agg({'accuracy': 'mean', 'recall': 'mean', 'precision': 'mean', 'f1_score': 'mean'}).reset_index()

,corr_threshold,num_monitored_features,accuracy,recall,precision,f1_score
0,0.1,1,0.544444,0.569444,0.502424,0.519599
1,0.1,2,0.547059,0.623413,0.508840,0.543653
2,0.1,4,0.540000,0.807143,0.541273,0.643335
3,0.1,9,0.450000,0.955000,0.420833,0.571537
4,0.1,19,0.000000,0.000000,0.000000,0.000000
5,0.2,1,0.344444,0.141667,0.300000,0.180241
6,0.2,2,0.382353,0.359524,0.336371,0.340191
7,0.2,4,0.413333,0.432143,0.379762,0.397484
8,0.2,9,0.460000,0.776667,0.461468,0.573643
9,0.2,19,0.000000,0.000000,0.000000,0.000000


In [323]:
# final_measurements.to_excel(f'measurements/final/{dataset_name}_results.xlsx', index=False)